# Prostate cANcer graDe Assessment (PANDA) Challenge

- Computer Vision / Image classification
- main challenge of this task is dealing with images of extremely high resolution and large areas of empty space. So, effective locating the areas of concern and zooming them in would be the key to reach high LB score.
- noisy data and train/test bias : solely relaying on CV was not really good strategy in this competition
- metric : QWK 

- Datasets : Train set(10616), Test set(940?) -> Train set labeled by 2 different institution(Karolinska, Radboud)

- Image size : not uniform

## (N) Iafoss : PANDA concat tile pooling starter [0.79 LB]
https://www.kaggle.com/iafoss/panda-concat-tile-pooling-starter-0-79-lb <br>
https://www.kaggle.com/iafoss/panda-16x128x128-tiles <br>
https://www.kaggle.com/c/prostate-cancer-grade-assessment/discussion/169205 <br>
https://github.com/iafoss/PANDA <br>

### Data preprocessing
- Tile extraction is based on my public pipeline with 128x128x128 tiles from intermediate resolution layer
- Label noise removal(removal of the uncertain examples from training set based on the out of fold predictions. I excluded ~1400 Redbound and 300 Karalinska data, so my clean training set contains about 8700 items) gives ~0.005 public and 0.01+ private LB boost
- tile cutout + tile selection augmentations
- more advanced tile selection could give ~0.004 boost at private LB on average (and the maximum private LB score of 0.941)
- LB was also not the best thing to trust because of severe noise, but some ppl tried to fit random seed as a hyperparameter. The right thing, in my opinion, in this competition was to find the balance between CV and LB, and trust to your intuition and the experience gained in the previous competitions.

### Modeling
- All my models are based on ResNeXt50, similar to my public kernel, with batch norm in the head replaced with Group-norm. The optimizer, best model selection based on CV, and other things are similar to my public kernel, and I was using 32-48 epochs, depending on the setup. In addition, I tried ResNet34, ResNeXt101, and EfficientNet, while all of them were performing worse. I think ResNet34 may be not capable enough for this task, while ResNeXt101 is too large to do training on my computer with sufficient bs.
- The submission that gave me the 11th place (0.930 private LB/0.917 public LB) is based on a majority voting ensemble of 8 models (4 fold) with 6 TTA.

## (D) fam_taro : 1st Place Solution [PND]
https://www.kaggle.com/c/prostate-cancer-grade-assessment/discussion/169143 <br>
https://github.com/kentaroy47/Kaggle-PANDA-1st-place-solution

### Data preprocessing
- Use Iafoss tile method & Qishen Ha bin label : How to make tiles? -> Split image by (tile_size, tile_size) -> Sort by pixel value for each tile -> Imgsize: ex. tile_size: 256, num_tile: 36 imsize: 256 x 6 = 1,536


### Modeling
- Model: EfficientNet B0-B1 with CELoss
- 1. Split kfold with image similarity : imgid (imghash similarity greater than 0.9) in same fold <br>
https://www.kaggle.com/yukkyo/imagehash-to-detect-duplicate-images-and-grouping
- 2. Training with original label (with noise)
- 3. Remove noise by prediction and original label gap(out of fold)
- 4. Re-train model without noise
- 5. Ensemble
- Tile mode augmentation (train & test)
- TTA(tile mode, hvflip, transpose)
- Convert label to bin : ex. 2 → [1, 1, 0, 0, 0], 4 → [1, 1, 1, 1, 0]


## Rohitsingh : PANDA - Inference (ENSEMBLE) trying various models
https://www.kaggle.com/rohitsingh9990/panda-inference-ensemble-trying-various-models

### Modeling
1. se_resnext50_32x4d / 3 folds ensemble / full image resized (224x224) / LB: 0.66
2. se_resnext50_32x4d / submitted on fold 0 / tiles of size (16x256x256) / CV: 0.7177 / LB: 0.67
3. se_resnext50_32x4d / 5 folds ensemble / tiles of size (16x256x256) / CV: [ fold0: 0.717, fold1: 0.674, fold2: 0.692, fold3: 0.704, fold4: 0.695 ] / LB: 0.70
4. efficientnet-B3 / 2 folds ensemble (fold 0 and fold 1) / tiles of size (16x256x256) / CV: [ fold0: 0.719, fold1: 0.726 ] / LB: 0.74
5. efficientnet-B3 /  5 folds ensemble / tiles of size (16x256x256) / CV: [ fold0: 0.719, fold1: 0.726, fold2: 0.729, fold3: 0.719, fold4: 0.732 ] / LB: 0.73
6. efficientnet-B3 / fold 0 / tiles of size (16x256x256) [image resized to 512x512] / CV: [ fold0: 0.749] / LB: 0.81
7. efficientnet-B3 / ensemble of all 5 folds / tiles of size (16x256x256) [image resized to 512x512] / CV: [ fold0: 0.749, fold1: 0.7509, fold2: 0.7592, fold3: 0.7362, fold4: 0.7676] / LB: 0.84
8. efficientnet-B3 / ensemble of all folds [0,1,2,4] / tiles of size (16x256x256) [image resized to 512x512] / CV: [ fold0: 0.749, fold1: 0.7509, fold2: 0.7592,fold4: 0.7676] / LB: 0.85

*Interesting findings <br>

- 1~3 : first used basic methods with resnext50, then used image preprocessing(tile method from above), and then used cross validation with 5 folds
- 4~5 : used efficientnetb3, it's better than resnext50. 5 folds worked poorer than 2 folds
- 6 : so tried 0 fold with image resizing, and it improved!
- 7~8 : maybe it was with the image resizing, not cv. so with image resizing, tried 5 folds again. and tried selectively making an ensemble